In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import os
os.chdir('..')

In [2]:
from langchain_openai import ChatOpenAI
from textlong.md import create_agent

In [3]:
app = create_agent(llm=ChatOpenAI(model_name="gpt-4-turbo"))

In [4]:
app.invoke({"task": "帮我创作一篇500字的都市修仙故事"})


 -------------------- 已有outline: 0

 -------------------- 已有detail: 0
反思: 由于目前没有写作提纲，我需要首先创建一个提纲来指导后续的写作过程。

思考: 为了开始写作，我首先需要一个清晰的提纲。这将帮助我组织故事的结构，并确保故事内容完整。

推理: 我将使用 create_outline 工具来生成一个都市修仙故事的写作提纲。这样做能确保故事有一个合理的流程和完整的结构。

计划: 我计划使用 create_outline 工具来生成提纲。我会提供一个简单的任务描述 "创作一篇都市修仙故事的提纲"，这将帮助工具理解我需要什么样的输出。

```
{"name": "create_outline", "args": {"task": "创作一篇都市修仙故事的提纲"}}
```>->>>
# 城市中的修仙者
## 介绍主角背景与现实生活（300字）
扩写要求：
- 描述主角的日常生活与职业
- 主角如何得知自己拥有修仙资质

## 主角遇到的神秘人物（200字）
扩写要求：
- 描述神秘人物的外貌和特征
- 神秘人物如何引导主角踏上修仙之路

## 主角的第一次修炼经历（300字）
扩写要求：
- 描述修炼的场所和过程
- 主角在修炼中遇到的挑战和感悟

## 主角的修仙伙伴与对手（250字）
扩写要求：
- 介绍至少一位修仙伙伴和一位对手
- 描述他们与主角的关系和互动

## 主角的第一场修仙战斗（400字）
扩写要求：
- 描述战斗的发生原因和地点
- 详细描写战斗过程和主角如何获胜

## 主角在都市中的双重生活（350字）
扩写要求：
- 描述主角如何平衡修仙与日常生活
- 主角在现实世界中如何应用修仙得到的智慧和力量

## 结尾：主角的修仙之路展望（200字）
扩写要求：
- 描述主角对未来修仙生活的期望
- 主角如何计划继续提高自己的修仙能力
<<<-<
 -------------------- 已有outline: 1

 -------------------- 已有detail: 0
1. 反思: 因为当前没有写作提纲，我们需要首先创建一个写作提纲。
2. 思考: 选择 create_outline 工具来生成写作提纲。
3. 推理: 我们需要一个提纲来指导故事的写作，确

NotImplementedError: 

In [ ]:
for x in app.stream({"task": "帮我创作一篇2000字的都市修仙故事"}):
    print(x)

In [ ]:
async for x in app.astream_events({"task": "帮我创作一篇2000字的都市修仙故事"}, version="v1"):
    kind = x["event"]
    if kind == "on_chat_model_stream":
        content = x["data"]["chunk"].content
        if content:
            print(content, end="", flush=True)
    else:
        # print(kind)
        pass

## 可视化

In [ ]:
%%capture --no-stderr
%pip install pygraphviz

In [ ]:
app.get_graph().print_ascii()

In [ ]:
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

display(
    Image(app.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API))
)

## RAG

In [ ]:
from langchain_zhipu import ChatZhipuAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain.agents import tool

from textlong.agents import create_tools_calling_executor
from textlong.qa import AskDocumentTool, create_qa_chain

query_document = Document(page_content="textlong 是一个基于AI的开源框架，用于生成超长文档")

db = FAISS.from_documents([query_document], OpenAIEmbeddings())

llm_zhipu = ChatZhipuAI()

chain = create_qa_chain(llm_zhipu, db.as_retriever())
qa_tool = AskDocumentTool(chain=(chain | StrOutputParser()), name="qa_chain")

In [ ]:
chain.invoke({"query": "textlong是什么"})

In [ ]:
chain.get_graph().print_ascii()

In [ ]:
from langchain.tools import tool
import random

# @tool(args_schema=WhereIsCatSchema)
@tool("WhereIsCatHidding")
def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底下", "在书架中", "在阳台"])

In [ ]:
# from langchain_openai import ChatOpenAI
# llm_zhipu = ChatZhipuAI()
tools = [qa_tool, where_is_cat_hiding]
app = create_tools_calling_executor(llm_zhipu, tools)

In [ ]:
for x in app.stream("讲个笑话给我?"):
    print(x)

In [ ]:
from langchain_core.messages import HumanMessage
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    verbose = True)

input = "请查询资料，告诉我textlong是什么？"
async for chunk in app.astream_events(input, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])        

In [ ]:
app.get_graph().print_ascii()

In [ ]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": chain},
    verbose = True
)

input = ["请查询资料，告诉我textlong是什么？"]
async for chunk in app.astream_events(input, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])

In [ ]:
app.get_graph().print_ascii()

In [ ]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": {"node": chain, "to": "agent"}},
    verbose = True)
app.get_graph().print_ascii()

In [ ]:
inputs = ["请查询资料，告诉我langchain_chinese是什么？"]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])

In [ ]:
inputs = [HumanMessage(content="猫在哪里？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

In [ ]:
inputs =  [HumanMessage(content="霍金的生日是哪一天？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

In [ ]:
inputs =  [HumanMessage(content="写一个关于程序员的笑话")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])